# Introduction

In this notebook we demonstrate the use of **LDA (Latent Dirichlet Allocation)** generative statistical model for Information Retrieval technique to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each test case content as an entire document that must be returned to the query made

## Import Libraries

In [2]:
import sys
if '../..' not in sys.path:
    sys.path.append('../..')

import pandas as pd
import numpy as np

from dit.divergences import jensen_shannon_divergence

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from modules.utils import plots
from modules.utils import firefox_dataset_p1 as fd
from modules.utils import tokenizers as tok
from modules.utils import aux_functions
from modules.utils import model_evaluator as m_eval

from modules.models.lda import LDA
from modules.models.model_hyperps import LDA_Model_Hyperp

from IPython.display import display

import warnings; warnings.simplefilter('ignore')

## Load Dataset

In [3]:
test_cases_df = fd.read_testcases_df()
bug_reports_df = fd.read_bugreports_df()

corpus = test_cases_df.tc_desc
query = bug_reports_df.br_desc

test_cases_names = test_cases_df.tc_name
bug_reports_names = bug_reports_df.br_name

orc = fd.read_trace_df()

TestCases.shape: (207, 12)
BugReports.shape: (35336, 18)
Oracle.shape: (207, 35316)


## Evaluate Recovering Efficiency

In order to evaluate the efficiency of the algorithm tested (LSI), we use common metrics applied in the field of IR:

    * Precision
    * Recall
    * F1-score

### Analysis with Default Values of LDA Model

In [6]:
model = LDA()
model.recover_links(corpus, query, use_cases_names, bug_reports_names)
evaluator = m_eval.ModelEvaluator(orc.oracle, model)
evaluator.evaluate_model(verbose=True)

{'Measures': {'Mean FScore of LDA': 0.0,
              'Mean Precision of LDA': 0.0,
              'Mean Recall of LDA': 0.0},
 'Setup': [{'Name': 'LDA'},
           {'Similarity Measure and Minimum Threshold': ('jsd', 0.3)},
           {'Top Value': 3},
           {'LDA Model': {'batch_size': 128,
                          'doc_topic_prior': None,
                          'evaluate_every': -1,
                          'learning_decay': 0.7,
                          'learning_method': None,
                          'learning_offset': 10.0,
                          'max_doc_update_iter': 100,
                          'max_iter': 10,
                          'mean_change_tol': 0.001,
                          'n_components': 10,
                          'n_jobs': -1,
                          'n_topics': None,
                          'perp_tol': 0.1,
                          'random_state': None,
                          'topic_word_prior': None,
                          'to

## Running LDA Model with Different Types of Oracles

### Strong and Weak Links Datasets

In [3]:
br_tc_strong_df = pd.read_csv('../../data/mozilla_firefox_v2/firefoxDataset/oracle/output/BR_TC_Strong.csv')
br_tc_weak_df = pd.read_csv('../../data/mozilla_firefox_v2/firefoxDataset/oracle/output/BR_TC_Weak.csv')
br_tc_mix_df = pd.read_csv('../../data/mozilla_firefox_v2/firefoxDataset/oracle/output/BR_TC_Mix.csv')

print(br_tc_strong_df.shape)
print(br_tc_weak_df.shape)
print(br_tc_mix_df.shape)

(10, 4)
(10, 4)
(10, 4)


### Define **run_lda_model()** Function

In [4]:
def run_lda_model(selected_tcs, selected_brs):
    tcs_df = test_cases_df[test_cases_df.tc_name.isin(selected_tcs)]
    brs_df = bug_reports_df[bug_reports_df.br_name.isin(selected_brs)]

    corpus_subset = tcs_df.tc_desc
    query_subset = brs_df.br_desc
    testcases_names_subset = tcs_df.tc_name
    bug_reports_names_subset = brs_df.br_name
    orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

    print('TestCases Subset Shape: {}'.format(tcs_df.shape))
    print('BugReports Subset Shape: {}'.format(brs_df.shape))
    print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

    lda_hyperp = {
        LDA_Model_Hyperp.TOP.value : 5,
        LDA_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : ('cosine',.75),
        LDA_Model_Hyperp.LDA_MODEL_N_COMPONENTS.value: 50,
        LDA_Model_Hyperp.LDA_MODEL_RANDOM_STATE.value : 2,
        LDA_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: (1,1),
        LDA_Model_Hyperp.VECTORIZER.value : TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True),
        LDA_Model_Hyperp.VECTORIZER_TOKENIZER.value : tok.PorterStemmerBased_Tokenizer() 
    }

    lda_model = LDA(**lda_hyperp)
    lda_model.set_name('LDA_Model_0')
    lda_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)

    print("\nModel Evaluation -------------------------------------------")
    evaluator = m_eval.ModelEvaluator(orc_subset_df, lda_model)
    evaluator.evaluate_model(verbose=True)
    
    print("\n\nTraceLinks Matrix --------------------------------------")
    display(aux_functions.highlight_df(lda_model.get_trace_links_df()))

    print("\n\nOracle -----------------------------------------")
    display(aux_functions.highlight_df(orc_subset_df))

### Oracle with Strong Links Only

In [5]:
selected_tcs = ['TC_{}_TRG'.format(tc_num) for tc_num in br_tc_strong_df.TC.values]
selected_brs = ['BR_{}_SRC'.format(bg_num) for bg_num in br_tc_strong_df.BR.values]

run_lda_model(selected_tcs, selected_brs)

TestCases Subset Shape: (10, 10)
BugReports Subset Shape: (6, 12)
Oracle Subset Shape: (10, 6)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LDA_Model_0': 0.5046296296296297,
              'Mean Precision of LDA_Model_0': 0.7666666666666666,
              'Mean Recall of LDA_Model_0': 0.4444444444444444},
 'Setup': [{'Name': 'LDA_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'LDA Model': {'batch_size': 128,
                          'doc_topic_prior': None,
                          'evaluate_every': -1,
                          'learning_decay': 0.7,
                          'learning_method': 'batch',
                          'learning_offset': 10.0,
                          'max_doc_update_iter': 100,
                          'max_iter': 10,
                          'mean_change_tol': 0.001,
                          'n_components': 50,
               

br_name,BR_1298575_SRC,BR_1313805_SRC,BR_1320658_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,,
TC_118_TRG,1,1,1,1,0,0
TC_120_TRG,0,0,0,0,0,0
TC_121_TRG,1,1,0,0,0,0
TC_143_TRG,0,0,0,0,0,1
TC_155_TRG,1,1,1,1,0,0
TC_172_TRG,1,1,1,1,0,0
TC_181_TRG,1,1,1,1,0,0
TC_183_TRG,0,0,1,1,0,0
TC_196_TRG,0,0,0,0,1,0




Oracle -----------------------------------------


,BR_1298575_SRC,BR_1313805_SRC,BR_1320658_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,,
TC_118_TRG,1,0,0,0,0,0
TC_120_TRG,1,0,0,0,0,0
TC_121_TRG,1,0,0,0,0,0
TC_143_TRG,0,1,1,1,1,1
TC_155_TRG,0,1,1,1,1,1
TC_172_TRG,0,1,1,1,1,1
TC_181_TRG,0,1,1,1,1,1
TC_183_TRG,0,1,1,1,1,1
TC_196_TRG,0,1,1,1,1,1


### Oracle with Weak Links Only

In [6]:
selected_tcs = ['TC_{}_TRG'.format(tc_num) for tc_num in br_tc_weak_df.TC.values]
selected_brs = ['BR_{}_SRC'.format(bg_num) for bg_num in br_tc_weak_df.BR.values]

run_lda_model(selected_tcs, selected_brs)

TestCases Subset Shape: (10, 10)
BugReports Subset Shape: (5, 12)
Oracle Subset Shape: (10, 5)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LDA_Model_0': 0.4238095238095238,
              'Mean Precision of LDA_Model_0': 0.44000000000000006,
              'Mean Recall of LDA_Model_0': 0.5571428571428572},
 'Setup': [{'Name': 'LDA_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'LDA Model': {'batch_size': 128,
                          'doc_topic_prior': None,
                          'evaluate_every': -1,
                          'learning_decay': 0.7,
                          'learning_method': 'batch',
                          'learning_offset': 10.0,
                          'max_doc_update_iter': 100,
                          'max_iter': 10,
                          'mean_change_tol': 0.001,
                          'n_components': 50,
              

br_name,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,1,1,1,1,1
TC_75_TRG,0,0,0,0,0
TC_105_TRG,1,1,1,1,1
TC_154_TRG,0,0,0,0,0
TC_155_TRG,0,0,0,0,1
TC_174_TRG,0,0,0,0,0
TC_196_TRG,1,1,1,1,0
TC_197_TRG,1,1,1,1,0
TC_200_TRG,0,0,0,0,1




Oracle -----------------------------------------


,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,1,0,0,0,0
TC_75_TRG,0,1,0,0,0
TC_105_TRG,0,1,0,0,0
TC_154_TRG,0,0,1,1,1
TC_155_TRG,0,0,1,1,1
TC_174_TRG,0,0,1,1,1
TC_196_TRG,0,0,1,1,1
TC_197_TRG,0,0,1,1,1
TC_200_TRG,0,0,1,1,1


### Oracle with Mixed Links (Strong and Weak)

In [7]:
selected_tcs = ['TC_{}_TRG'.format(tc_num) for tc_num in br_tc_mix_df.TC.values]
selected_brs = ['BR_{}_SRC'.format(bg_num) for bg_num in br_tc_mix_df.BR.values]

run_lda_model(selected_tcs, selected_brs)

TestCases Subset Shape: (10, 10)
BugReports Subset Shape: (5, 12)
Oracle Subset Shape: (10, 5)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LDA_Model_0': 0.48,
              'Mean Precision of LDA_Model_0': 0.7666666666666666,
              'Mean Recall of LDA_Model_0': 0.4666666666666666},
 'Setup': [{'Name': 'LDA_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'LDA Model': {'batch_size': 128,
                          'doc_topic_prior': None,
                          'evaluate_every': -1,
                          'learning_decay': 0.7,
                          'learning_method': 'batch',
                          'learning_offset': 10.0,
                          'max_doc_update_iter': 100,
                          'max_iter': 10,
                          'mean_change_tol': 0.001,
                          'n_components': 50,
                          'n_

br_name,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,1,0,0,0,0
TC_105_TRG,0,0,0,0,0
TC_118_TRG,0,0,0,0,0
TC_121_TRG,0,1,0,0,0
TC_143_TRG,0,1,0,1,1
TC_154_TRG,1,0,0,0,0
TC_155_TRG,1,0,0,0,0
TC_172_TRG,0,0,1,0,0
TC_174_TRG,0,0,1,0,0




Oracle -----------------------------------------


,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,1,0,0,0,0
TC_105_TRG,0,1,0,0,0
TC_118_TRG,0,1,0,0,0
TC_121_TRG,0,1,0,0,0
TC_143_TRG,0,0,1,1,1
TC_154_TRG,0,0,1,1,1
TC_155_TRG,0,0,1,1,1
TC_172_TRG,0,0,1,1,1
TC_174_TRG,0,0,1,1,1


### General Test

In [7]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

selected_bugreports = bugreports_subset_df.br_name
run_lda_model(selected_testcases, selected_bugreports)


TestCases Subset Shape: (6, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (6, 15)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LDA_Model_0': 0.15333333333333332,
              'Mean Precision of LDA_Model_0': 0.26666666666666666,
              'Mean Recall of LDA_Model_0': 0.1111111111111111},
 'Setup': [{'Name': 'LDA_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'LDA Model': {'batch_size': 128,
                          'doc_topic_prior': None,
                          'evaluate_every': -1,
                          'learning_decay': 0.7,
                          'learning_method': 'batch',
                          'learning_offset': 10.0,
                          'max_doc_update_iter': 100,
                          'max_iter': 10,
                          'mean_change_tol': 0.001,
                          'n_components': 50,
             

br_name,BR_1268934_SRC,BR_1282551_SRC,BR_1291175_SRC,BR_1299787_SRC,BR_1418983_SRC,BR_1432520_SRC,BR_1436749_SRC,BR_1443632_SRC,BR_1443754_SRC,BR_1450216_SRC,BR_1461828_SRC,BR_1463274_SRC,BR_1463735_SRC,BR_1497738_SRC,BR_1513270_SRC
tc_name,,,,,,,,,,,,,,,
TC_13_TRG,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1
TC_14_TRG,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1
TC_15_TRG,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0
TC_16_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
TC_17_TRG,1,0,0,1,1,1,1,0,1,0,1,0,0,0,1
TC_18_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1




Oracle -----------------------------------------


,BR_1268934_SRC,BR_1282551_SRC,BR_1291175_SRC,BR_1299787_SRC,BR_1418983_SRC,BR_1432520_SRC,BR_1436749_SRC,BR_1443632_SRC,BR_1443754_SRC,BR_1450216_SRC,BR_1461828_SRC,BR_1463274_SRC,BR_1463735_SRC,BR_1497738_SRC,BR_1513270_SRC
tc_name,,,,,,,,,,,,,,,
TC_13_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_14_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_17_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_18_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
